In [4]:
import cv2
import numpy as np
from PIL import Image
import copy

In [ ]:
cap = cv2.VideoCapture(1)
count = 0
font = cv2.FONT_HERSHEY_SIMPLEX
length = 5
frames = [None] * length

In [ ]:
while True:
    ret, frame = cap.read()
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    cv2.imshow('frame', frame)
    key = cv2.waitKey(1)
    if key == 27:
        break
cap.release()
cv2.destroyAllWindows()

In [3]:
while True:
        
    out = cv2.VideoWriter('output.avi', -1, 20.0, (640,480))
    
    ret, frame = cap.read()
    
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    frame_np = np.array(frame)
    backup = np.zeros(frame.shape)
    
    if count < length:
        frames[count] = frame_np
    else:
        backup = frames[count % length]
        frames[count % length] = frame_np
        
    frame_np = frame_np.astype('int32')
    backup = backup.astype('int32')
    diff = np.abs(frame_np - backup)
    diff = diff.astype('uint8')
    frame_np = frame_np.astype('uint8')
    backup = backup.astype('uint8')
    
    
    out.write(diff)
    
    cv2.imshow('diff abs', diff)
    cv2.imshow('frame', frame_np)
    print (white_pixel_counter)
    count += 1
    key = cv2.waitKey(1)
    if key == 27:
        break
cap.release()
out.release()
cv2.destroyAllWindows()

error: OpenCV(4.1.0) /io/opencv/modules/imgproc/src/color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'


In [1]:
import numpy as np
import cv2
import copy

def findWhitePixels(diff, side_length, threshold, height, width):
    for i in range(0, height - threshold):
        for j in range(0, width - threshold):
            diff_m = (diff[i:(i + side_length)][j:(j + side_length)]) >= 150
            

cap = cv2.VideoCapture(0)
count = 0
font = cv2.FONT_HERSHEY_SIMPLEX
length = 5
frames = [None] * length
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('output15.avi', fourcc, 10.0, (640,480), 0)

while True:
    ret, frame = cap.read()
    height, width, t = frame.shape
    if ret==True:
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        frame_np = np.array(frame)
        backup = np.zeros(frame.shape)
        if count == 0:
            coords = input('size of he square ')
            
        if count < length:
            frames[count] = frame_np
        else:
            backup = frames[count % length]
            frames[count % length] = frame_np

        frame_np = frame_np.astype('int32')
        backup = backup.astype('int32')
        diff = np.abs(frame_np - backup)
        diff = diff.astype('uint8')
        frame_np = frame_np.astype('uint8')
        backup = backup.astype('uint8')


        out.write(diff)
        print (coords)
        #diff_c = copy.deepcopy(diff[coords[0]:coords[2], coords[1]:coords[3]])
        diff_m = (diff >= 150)
        white_pixel_counter = np.sum(diff_m)
        print (white_pixel_counter)
        
            
        cv2.imshow('frame',diff)
        count += 1
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    else:
        break
cap.release()

out.release()

cv2.destroyAllWindows()

write coordinates for scanner: 0, 0, 100, 100
0, 0, 100, 100
58508
0, 0, 100, 100
59005
0, 0, 100, 100
59131
0, 0, 100, 100
59478
0, 0, 100, 100
63166
0, 0, 100, 100
2346
0, 0, 100, 100
2170
0, 0, 100, 100
2247
0, 0, 100, 100
2378
0, 0, 100, 100
2721
0, 0, 100, 100
0
0, 0, 100, 100
0
0, 0, 100, 100
0
0, 0, 100, 100
0
0, 0, 100, 100
0
0, 0, 100, 100
0
0, 0, 100, 100
0
0, 0, 100, 100
0
0, 0, 100, 100
0
0, 0, 100, 100
0
0, 0, 100, 100
0
0, 0, 100, 100
0
0, 0, 100, 100
0
0, 0, 100, 100
0
0, 0, 100, 100
0
0, 0, 100, 100
0
0, 0, 100, 100
0
0, 0, 100, 100
1
0, 0, 100, 100
16
0, 0, 100, 100
48
0, 0, 100, 100
62
0, 0, 100, 100
76
0, 0, 100, 100
71
0, 0, 100, 100
66
0, 0, 100, 100
59
0, 0, 100, 100
32
0, 0, 100, 100
21
0, 0, 100, 100
19
0, 0, 100, 100
0
0, 0, 100, 100
0
0, 0, 100, 100
0
0, 0, 100, 100
0
0, 0, 100, 100
0
0, 0, 100, 100
0
0, 0, 100, 100
0
0, 0, 100, 100
0
0, 0, 100, 100
0
0, 0, 100, 100
11
0, 0, 100, 100
0
0, 0, 100, 100
0
0, 0, 100, 100
0
0, 0, 100, 100
0
0, 0, 100, 100
0
0, 0, 1

0, 0, 100, 100
0
0, 0, 100, 100
0
0, 0, 100, 100
0
0, 0, 100, 100
0
0, 0, 100, 100
0
0, 0, 100, 100
0
0, 0, 100, 100
0
0, 0, 100, 100
0
0, 0, 100, 100
0
0, 0, 100, 100
0
0, 0, 100, 100
0
0, 0, 100, 100
0
0, 0, 100, 100
0
0, 0, 100, 100
0
0, 0, 100, 100
0
0, 0, 100, 100
0
0, 0, 100, 100
0
0, 0, 100, 100
0
0, 0, 100, 100
0
0, 0, 100, 100
0
0, 0, 100, 100
0
0, 0, 100, 100
0
0, 0, 100, 100
0
0, 0, 100, 100
0
0, 0, 100, 100
0
0, 0, 100, 100
0
0, 0, 100, 100
0
0, 0, 100, 100
0
0, 0, 100, 100
0
0, 0, 100, 100
0
0, 0, 100, 100
0
0, 0, 100, 100
0
0, 0, 100, 100
0
0, 0, 100, 100
0
0, 0, 100, 100
0
0, 0, 100, 100
0
0, 0, 100, 100
0
0, 0, 100, 100
0
0, 0, 100, 100
0
0, 0, 100, 100
0
0, 0, 100, 100
0
0, 0, 100, 100
0
0, 0, 100, 100
0
0, 0, 100, 100
0
0, 0, 100, 100
0
0, 0, 100, 100
0
0, 0, 100, 100
0
0, 0, 100, 100
0
0, 0, 100, 100
0
0, 0, 100, 100
0
0, 0, 100, 100
0
0, 0, 100, 100
0
0, 0, 100, 100
0
0, 0, 100, 100
0
0, 0, 100, 100
0
0, 0, 100, 100
0
0, 0, 100, 100
0
0, 0, 100, 100
0
0, 0, 100, 100

In [ ]:
cv2.imshow('1', diff[150:200, 150:200])
cv2.imshow('2', diff_c)
cv2.imshow('3', frame_np)
cv2.waitKey(0)
cv2.destroyAllWindows()
print (np.average(diff))

In [ ]:
out = cv2.VideoWriter('output.avi', -1, 20.0, (640,480))

while(cap.isOpened()):
    ret, frame = cap.read()

    out.write(frame)

    cv2.imshow('frame',frame)
else:
    break

# Release everything if job is finished
cap.release()
out.release()
cv2.destroyAllWindows()

In [ ]:
backup_cutout = backup[150:200, 150:200]
frame_np_cutout = frame_np[150:200, 150:200]
diff_cutout = diff[150:200, 150:200]
#print (diff_cutout.dtype)
print (diff_cutout)
print ('\n')
print (backup_cutout)
print ('\n')
print(frame_np_cutout)
print ('\n')
print (frame_np_cutout - backup_cutout)

In [ ]:
while True:
    ret, frame = cap.read()
    a = np.array(frame)
    b = np.average(a)
    print (a.shape)
    cv2.imshow('frame', frame)
    key = cv2.waitKey(1)
    if key == 27:
        break
cap.release()
cv2.destroyAllWindows()

In [2]:
coords = input('write coordinates for scanner: ')
print (coords[1])

write coordinates for scanner: [0, 0, 10, 10]
0
